# Traditional Models

In [55]:
def load_tweets(path):
    with open(path, "rb") as f:
        import pickle
        e_tweets = pickle.load(f)
    return e_tweets

labels = load_tweets(path = "data/Sentiment_Labels.pkl")

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif

def process_tweets(tweets, labels, k = 20000, ngrams = (1, 2)):
    train, test, y_train, y_test = train_test_split(tweets, labels, train_size=0.8, test_size=0.2, random_state=42)
    
    vectorizer = CountVectorizer(ngram_range = ngrams, stop_words='english')
    train_bow = vectorizer.fit_transform(train)
    test_bow = vectorizer.transform(test)
    
    selector = SelectKBest(f_classif, k).fit(train_bow, y_train)
    train_topK_bow = selector.transform(train_bow).astype('float32')
    test_topK_bow = selector.transform(test_bow).astype('float32')
    
    tfidf_vectorizer = TfidfVectorizer(ngram_range = ngrams, stop_words='english')
    train_tfidf = tfidf_vectorizer.fit_transform(train)
    test_tfidf = tfidf_vectorizer.transform(test)
    
    selector_tfidf = SelectKBest(f_classif, k).fit(train_tfidf, y_train)
    train_topK_tfidf = selector_tfidf.transform(train_tfidf).astype('float32')
    test_topK_tfidf = selector.transform(test_tfidf).astype('float32')
    
    data = {}
    data['bow'] = train_bow, test_bow, train_topK_bow, test_topK_bow
    data['tfidf'] = train_tfidf, test_tfidf, train_topK_tfidf, test_topK_tfidf
    data['labels'] = y_train, y_test
    
    
    return data

In [59]:
clean_tweets = load_tweets(path = "data/clean_text.pkl")
clean_data =  process_tweets(clean_tweets, labels)
y_train, y_test = clean_data['labels']

In [64]:
train_bow, test_bow, train_topK_bow, test_topK_bow = clean_data['bow']
train_tfidf, test_tfidf, train_topK_tfidf, test_topK_tfidf = clean_data['tfidf']

print(f"BoW Train Features shape: {train_bow.shape}, BoW Test Features shape: {test_bow.shape} " )
print(f"TF-IDF Train Features shape: {train_tfidf.shape}, TF-IDF Test Features shape: {test_tfidf.shape} " )

BoW Train Features shape: (34753, 178008), BoW Test Features shape: (8689, 178008) 
TF-IDF Train Features shape: (34753, 178008), TF-IDF Test Features shape: (8689, 178008) 


In [65]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
 
def fit_and_metrics(classifier, train, test):
    clf = classifier.fit(train, y_train)
    y_predicted = clf.predict(test)
    
    
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

# Logistic Regression

In [37]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial', max_iter=500)

In [38]:
accuracy_bow_lr, precision_bow_lr, recall_bow_lr, f1_bow_lr = fit_and_metrics(lr, train_bow, test_bow)

accuracy_k_bow_lr, precision_k_bow_lr, recall_k_bow_lr, f1_k_bow_lr = fit_and_metrics(lr, train_topK_bow,
                                                                                      test_topK_bow)
print("BoW Model: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_bow_lr, 
                                                                                  precision_bow_lr,
                                                                                  recall_bow_lr, 
                                                                                  f1_bow_lr))

print("Top-K BoW: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_k_bow_lr, 
                                                                                  precision_k_bow_lr,
                                                                                  recall_k_bow_lr, 
                                                                                  f1_k_bow_lr))

BoW Model: accuracy = 0.844, precision = 0.846, recall = 0.844, f1 = 0.843
Top-K BoW: accuracy = 0.852, precision = 0.851, recall = 0.852, f1 = 0.851


In [39]:
accuracy_tfidf_lr, precision_tfidf_lr, recall_tfidf_lr, f1_tfidf_lr = metrics(lr, train_tfidf, test_tfidf)

accuracy_k_tfidf_lr, precision_k_tfidf_lr, recall_k_tfidf_lr, f1_k_tfidf_lr = fit_and_metrics(lr, train_topK_tfidf,
                                                                                      test_topK_tfidf)

print("TF-IDF Model: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf_lr, 
                                                                                  precision_tfidf_lr,
                                                                                  recall_tfidf_lr, 
                                                                                  f1_tfidf_lr))

print("Top-K TF-IDF: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_k_tfidf_lr, 
                                                                                  precision_k_tfidf_lr,
                                                                                  recall_k_tfidf_lr, 
                                                                                  f1_k_tfidf_lr))

TF-IDF Model: accuracy = 0.813, precision = 0.810, recall = 0.813, f1 = 0.807
Top-K TF-IDF: accuracy = 0.798, precision = 0.795, recall = 0.798, f1 = 0.789


# SVM

In [66]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier()

In [67]:
accuracy_bow_svm, precision_bow_svm, recall_bow_svm, f1_bow_svm = fit_and_metrics(svm, train_bow, test_bow)

accuracy_k_bow_svm, precision_k_bow_svm, recall_k_bow_svm, f1_k_bow_svm = fit_and_metrics(svm, train_topK_bow,
                                                                                      test_topK_bow)

print("BoW Model: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_bow_svm, 
                                                                                  precision_bow_svm, 
                                                                                  recall_bow_svm, 
                                                                                  f1_bow_svm))

print("Top-K BoW: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_k_bow_svm, 
                                                                                  precision_k_bow_svm,
                                                                                  recall_k_bow_svm, 
                                                                                  f1_k_bow_svm))

BoW Model: accuracy = 0.847, precision = 0.851, recall = 0.847, f1 = 0.847
Top-K BoW: accuracy = 0.863, precision = 0.864, recall = 0.863, f1 = 0.863


In [42]:
accuracy_tfidf_svm, precision_tfidf_svm, recall_tfidf_svm, f1_tfidf_svm = fit_and_metrics(svm, train_tfidf, test_tfidf)

accuracy_k_tfidf_svm, precision_k_tfidf_svm, recall_k_tfidf_svm, f1_k_tfidf_svm = fit_and_metrics(svm,train_topK_tfidf,
                                                                                                  test_topK_tfidf)
print("TF-IDF Model: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf_svm, 
                                                                                     precision_tfidf_svm, 
                                                                                     recall_tfidf_svm, 
                                                                                     f1_tfidf_svm))
print("Top-K TF-IDF: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_k_tfidf_svm, 
                                                                                  precision_k_tfidf_svm,
                                                                                  recall_k_tfidf_svm, 
                                                                                  f1_k_tfidf_svm))

TF-IDF Model: accuracy = 0.786, precision = 0.787, recall = 0.786, f1 = 0.770
Top-K TF-IDF: accuracy = 0.778, precision = 0.782, recall = 0.778, f1 = 0.758


# Random Forest

In [70]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [71]:
accuracy_bow_rf, precision_bow_rf, recall_bow_rf, f1_bow_rf = fit_and_metrics(rf, train_bow, test_bow)

accuracy_k_bow_rf, precision_k_bow_rf, recall_k_bow_rf, f1_k_bow_rf = fit_and_metrics(rf, train_topK_bow,
                                                                                      test_topK_bow)

print("BoW Model: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_bow_rf, 
                                                                                  precision_bow_rf, 
                                                                                  recall_bow_rf, 
                                                                                  f1_bow_rf))

print("Top-K BoW: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_k_bow_rf, 
                                                                                  precision_k_bow_rf,
                                                                                  recall_k_bow_rf, 
                                                                                  f1_k_bow_rf))

BoW Model: accuracy = 0.810, precision = 0.817, recall = 0.810, f1 = 0.809
Top-K BoW: accuracy = 0.807, precision = 0.806, recall = 0.807, f1 = 0.803


In [72]:
accuracy_tfidf_rf, precision_tfidf_rf, recall_tfidf_rf, f1_tfidf_rf = fit_and_metrics(rf, train_tfidf, test_tfidf)

accuracy_k_tfidf_rf, precision_k_tfidf_rf, recall_k_tfidf_rf, f1_k_tfidf_rf = fit_and_metrics(rf, train_topK_tfidf,
                                                                                      test_topK_tfidf)

print("TF-IDF Model: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf_rf, 
                                                                                     precision_tfidf_rf, 
                                                                                     recall_tfidf_rf, 
                                                                                     f1_tfidf_rf))

print("Top-K TF-IDF: accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_k_tfidf_rf, 
                                                                                  precision_k_tfidf_rf,
                                                                                  recall_k_tfidf_rf, 
                                                                                  f1_k_tfidf_rf))

TF-IDF Model: accuracy = 0.806, precision = 0.811, recall = 0.806, f1 = 0.804
Top-K TF-IDF: accuracy = 0.242, precision = 0.415, recall = 0.242, f1 = 0.176


In [76]:
Classifiers = [
    LogisticRegression(multi_class='multinomial', max_iter=500),
    SGDClassifier(),
    RandomForestClassifier()]

# Stemmed Tweets

In [82]:
stem_tweets = load_tweets(path = "data/stemmed_text.pkl")
stem_data =  process_tweets(stem_tweets, labels)
stem_train, stem_test = stem_data['bow'][0], stem_data['bow'][1]
y_train, y_test = stem_data['labels']

In [87]:
for clf in Classifiers:
    fit = clf.fit(stem_train, y_train)
    pred = fit.predict(stem_test) 
    acc = accuracy_score(y_test, pred)
    print("Accuracy of " + clf.__class__.__name__ + " is " + str(acc))

Accuracy of LogisticRegression is 0.8346184831396017
Accuracy of SGDClassifier is 0.8362297157325354
Accuracy of RandomForestClassifier is 0.8037748877891587


# Lemmatized Tweets

In [85]:
lem_tweets = load_tweets(path = "data/lemmatized_text.pkl")
lem_data =  process_tweets(lem_tweets, labels)
lem_train, lem_test = lem_data['bow'][0], lem_data['bow'][1]
y_train, y_test = lem_data['labels']

In [86]:
for clf in Classifiers:
    fit = clf.fit(lem_train, y_train)
    pred = fit.predict(lem_test) 
    acc = accuracy_score(y_test, pred)
    print("Accuracy of " + clf.__class__.__name__ + " is " + str(acc))

Accuracy of LogisticRegression is 0.8394521809184026
Accuracy of SGDClassifier is 0.843595350443089
Accuracy of RandomForestClassifier is 0.8007825986879963
